In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/MBX/quip_interface/test


In [6]:
from collections import OrderedDict

In [7]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [8]:
version_list=["mbx_pbc","mbx_pbc_no_mww_pip","mbx_pbc_no_www_mww","mbx_pbc_no_ww_mw_www_mww",
              "mbx_pbc_no_ww_mw","mbx_pbc_no_ww","mbx_pbc_no_mw"]

In [9]:
dict_version_d_virial=OrderedDict([(version,
                                    OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                   for version in version_list])

In [10]:
dict_version_model_virial=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [11]:
def calculate_numerical_virial_diag(pot, in_atoms, d=1e-2):
    ## modified from the function calculate_numerical_stress() at ase/calculators/calculator.py 
    """Calculate numerical stress using finite difference."""

    atoms=in_atoms.copy()
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eplus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eminus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
    atoms.set_cell(cell, scale_atoms=True)
    virial= (-stress*atoms.get_volume())
    return virial

In [13]:
this_root="/home/es732/programs/MBX/quip_interface/test/"

In [14]:
this_struct="dimer_pbc_test.xyz"
this_at_tmp=quippy.Atoms(this_root+this_struct)
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

In [64]:
this_at_tmp.get_distance(0,5)

3.5400759766759005

## diagonal_virial=F, no PIPs

In [15]:
this_version=version_list[3]
print this_version

mbx_pbc_no_ww_mw_www_mww


In [16]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [17]:
at= this_at_tmp.copy()

In [18]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1.43232229 -0.20591496  1.22131836]
 [-0.20591496 -1.72251679  0.46415493]
 [ 1.22131836  0.46415493 -2.34800644]]


In [19]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no 3b PIPs

In [20]:
this_version=version_list[2]
print this_version

mbx_pbc_no_www_mww


In [21]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [22]:
at= this_at_tmp.copy()

In [23]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1.41801525 -0.19417328  1.21840974]
 [-0.19417328 -1.70201764  0.46224119]
 [ 1.21840974  0.46224119 -2.33576414]]


In [24]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, full MBX

In [55]:
this_version=version_list[0]
print this_version

mbx_pbc


In [56]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [57]:
at= this_at_tmp.copy()

In [58]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1.41801525 -0.19417328  1.21840974]
 [-0.19417328 -1.70201764  0.46224119]
 [ 1.21840974  0.46224119 -2.33576414]]


In [60]:
for tmp_d in [0.1,0.01,0.001,0.0001,0.00001, 0.000001]: #, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no 2b PIPs

In [30]:
this_version=version_list[4]
print this_version

mbx_pbc_no_ww_mw


In [31]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [32]:
at= this_at_tmp.copy()

In [33]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1.43232229 -0.20591496  1.22131836]
 [-0.20591496 -1.72251679  0.46415493]
 [ 1.22131836  0.46415493 -2.34800644]]


In [34]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [35]:
for tmp_d in [0.00001,0.000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no ww 2b PIPs

In [36]:
this_version=version_list[5]
print this_version

mbx_pbc_no_ww


In [37]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [38]:
at= this_at_tmp.copy()

In [39]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1.41801525 -0.19417328  1.21840974]
 [-0.19417328 -1.70201764  0.46224119]
 [ 1.21840974  0.46224119 -2.33576414]]


In [40]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [41]:
for tmp_d in [0.00001,0.000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no mw 2b PIPs

In [42]:
this_version=version_list[6]
print this_version

mbx_pbc_no_mw


In [43]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [44]:
at= this_at_tmp.copy()

In [45]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1.43232229 -0.20591496  1.22131836]
 [-0.20591496 -1.72251679  0.46415493]
 [ 1.22131836  0.46415493 -2.34800644]]


In [46]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [47]:
for tmp_d in [0.00001,0.000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [48]:
version_list

['mbx_pbc',
 'mbx_pbc_no_mww_pip',
 'mbx_pbc_no_www_mww',
 'mbx_pbc_no_ww_mw_www_mww',
 'mbx_pbc_no_ww_mw',
 'mbx_pbc_no_ww',
 'mbx_pbc_no_mw']

## Errors in derivative virial / Virial from model (only diagonal)

### full model

In [61]:
this_version="mbx_pbc"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.0005575  -0.00173787 -0.00100369]
d = 0.001
[-5.57626378e-06 -1.73759756e-05 -1.00379589e-05]
d = 0.0001
[-5.56147834e-08 -1.65912594e-07 -1.18610387e-07]
d = 1e-05
[-8.20622058e-08  1.52606284e-07  1.56066000e-08]
d = 1e-06
[-2.23213218e-06  3.63523234e-07 -7.92234396e-07]


### model without PIPs

In [50]:
this_version="mbx_pbc_no_ww_mw_www_mww"
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00051398 -0.00172035 -0.00099964]
d = 0.001
[-5.14114554e-06 -1.72009058e-05 -9.99751432e-06]
d = 0.0001
[-5.13546360e-08 -1.64194957e-07 -1.18161675e-07]


### model without 2B PIPs

In [51]:
this_version="mbx_pbc_no_ww_mw"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00051398 -0.00172035 -0.00099964]
d = 0.001
[-5.14114554e-06 -1.72009058e-05 -9.99751432e-06]
d = 0.0001
[-5.13546360e-08 -1.64194957e-07 -1.18161675e-07]
d = 1e-05
[-8.12211124e-08  1.50225397e-07  1.52266101e-08]
d = 1e-06
[-2.21144971e-06  3.56305919e-07 -7.84254712e-07]


### model without 3B PIPs

In [52]:
this_version="mbx_pbc_no_www_mww"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.0005575  -0.00173787 -0.00100369]
d = 0.001
[-5.57626378e-06 -1.73759756e-05 -1.00379589e-05]
d = 0.0001
[-5.56147834e-08 -1.65912594e-07 -1.18610387e-07]
d = 1e-05
[nan nan nan]
d = 1e-06
[nan nan nan]


### model without 2B ww PIPs

In [53]:
this_version="mbx_pbc_no_ww"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.0005575  -0.00173787 -0.00100369]
d = 0.001
[-5.57626378e-06 -1.73759756e-05 -1.00379589e-05]
d = 0.0001
[-5.56147834e-08 -1.65912594e-07 -1.18610387e-07]
d = 1e-05
[-8.20622058e-08  1.52606284e-07  1.56066000e-08]
d = 1e-06
[-2.23213218e-06  3.63523234e-07 -7.92234396e-07]


### model without 2B mw PIPs

In [54]:
this_version="mbx_pbc_no_mw"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00051398 -0.00172035 -0.00099964]
d = 0.001
[-5.14114554e-06 -1.72009058e-05 -9.99751432e-06]
d = 0.0001
[-5.13546360e-08 -1.64194957e-07 -1.18161675e-07]
d = 1e-05
[-8.12211124e-08  1.50225397e-07  1.52266101e-08]
d = 1e-06
[-2.21144971e-06  3.56305919e-07 -7.84254712e-07]
